In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:
!pip install kaggle

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'microsoft-catsvsdogs-dataset' dataset.
Path to dataset files: /kaggle/input/microsoft-catsvsdogs-dataset


In [7]:
import os

base_path = path  # from kagglehub
print(os.listdir(base_path))

['PetImages', 'readme[1].txt', 'MSR-LA - 3467.docx']


In [8]:
pet_path = os.path.join(base_path, "PetImages")
print(os.listdir(pet_path))

['Dog', 'Cat']


In [9]:
import shutil
import os

source_dir = pet_path  # this is PetImages
target_dir = "/content/dataset_raw"

shutil.copytree(source_dir, target_dir)

print("Dataset copied to writable directory.")


Dataset copied to writable directory.


In [10]:
from PIL import Image

def clean_images(folder):
    bad_images = 0
    for cls in ["Cat", "Dog"]:
        class_path = os.path.join(folder, cls)
        for img in os.listdir(class_path):
            img_path = os.path.join(class_path, img)

            # Skip non-image system files
            if not img.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            try:
                Image.open(img_path).verify()
            except:
                os.remove(img_path)
                bad_images += 1

    print(f"Removed {bad_images} corrupted images")

clean_images("/content/dataset_raw")

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Removed 2 corrupted images


In [15]:
!ls /content/dataset_raw/Cat | head
!ls /content/dataset_raw/Dog | head

0.jpg
10000.jpg
10001.jpg
10002.jpg
10003.jpg
10004.jpg
10005.jpg
10006.jpg
10007.jpg
10008.jpg
0.jpg
10000.jpg
10001.jpg
10002.jpg
10003.jpg
10004.jpg
10005.jpg
10006.jpg
10007.jpg
10008.jpg


In [16]:
import shutil
shutil.rmtree("/content/dataset", ignore_errors=True)
print("Old dataset removed")

Old dataset removed


In [17]:
import os

for split in ["train", "val"]:
    for cls in ["cat", "dog"]:
        os.makedirs(f"/content/dataset/{split}/{cls}", exist_ok=True)

print("New dataset folders created")

New dataset folders created


In [18]:
import os, random, shutil

SOURCE = "/content/dataset_raw"
TARGET = "/content/dataset"
VALID_EXT = (".jpg", ".jpeg", ".png")

def split_and_copy(class_name):
    src_dir = os.path.join(SOURCE, class_name.capitalize())
    images = [f for f in os.listdir(src_dir) if f.lower().endswith(VALID_EXT)]

    print(f"{class_name.upper()} images found:", len(images))

    random.shuffle(images)
    split_idx = int(0.8 * len(images))

    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    for img in train_imgs:
        shutil.copy(
            os.path.join(src_dir, img),
            os.path.join(TARGET, "train", class_name, img)
        )

    for img in val_imgs:
        shutil.copy(
            os.path.join(src_dir, img),
            os.path.join(TARGET, "val", class_name, img)
        )

split_and_copy("cat")
split_and_copy("dog")


CAT images found: 12499
DOG images found: 12499


In [19]:
!ls dataset/train/cat | head
!ls dataset/train/dog | head
!ls dataset/val/cat | head
!ls dataset/val/dog | head

0.jpg
10000.jpg
10001.jpg
10002.jpg
10003.jpg
10004.jpg
10006.jpg
10007.jpg
10008.jpg
10009.jpg
10000.jpg
10001.jpg
10002.jpg
10003.jpg
10004.jpg
10005.jpg
10006.jpg
10007.jpg
10008.jpg
1000.jpg
10005.jpg
1000.jpg
10010.jpg
10017.jpg
10018.jpg
10019.jpg
10021.jpg
10046.jpg
10052.jpg
10053.jpg
0.jpg
10009.jpg
10014.jpg
10015.jpg
10027.jpg
10030.jpg
10033.jpg
1003.jpg
10040.jpg
10053.jpg


In [22]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),   # required for ResNet
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder("dataset/train", transform=train_transform)
val_data = datasets.ImageFolder("dataset/val", transform=val_transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

print("Classes:", train_data.classes)


Classes: ['cat', 'dog']


In [23]:
from torchvision import datasets

train_data = datasets.ImageFolder("dataset/train", transform=train_transform)
val_data   = datasets.ImageFolder("dataset/val", transform=val_transform)

print("Classes:", train_data.classes)
print("Train samples:", len(train_data))
print("Val samples:", len(val_data))


Classes: ['cat', 'dog']
Train samples: 19998
Val samples: 5000


In [24]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 80.2MB/s]


In [25]:
for param in model.parameters():
    param.requires_grad = True


In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [27]:
def train_one_epoch(model, loader):
    model.train()
    total_loss, correct = 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = correct / len(loader.dataset)
    return avg_loss, accuracy

In [28]:
def validate(model, loader):
    model.eval()
    correct = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct += (outputs.argmax(1) == labels).sum().item()

    return correct / len(loader.dataset)


In [29]:
import torch.nn as nn
import torch

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=0.001
)


In [30]:
epochs = 15

for epoch in range(epochs):
    loss, train_acc = train_one_epoch(model, train_loader)
    val_acc = validate(model, val_loader)

    print(
        f"Epoch {epoch+1}/{epochs} | "
        f"Loss: {loss:.4f} | "
        f"Train Acc: {train_acc:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )

Epoch 1/15 | Loss: 0.1782 | Train Acc: 0.9285 | Val Acc: 0.9212
Epoch 2/15 | Loss: 0.1258 | Train Acc: 0.9495 | Val Acc: 0.9392
Epoch 3/15 | Loss: 0.1099 | Train Acc: 0.9572 | Val Acc: 0.9632
Epoch 4/15 | Loss: 0.0941 | Train Acc: 0.9638 | Val Acc: 0.9486
Epoch 5/15 | Loss: 0.0888 | Train Acc: 0.9643 | Val Acc: 0.9668
Epoch 6/15 | Loss: 0.0829 | Train Acc: 0.9660 | Val Acc: 0.9624
Epoch 7/15 | Loss: 0.0772 | Train Acc: 0.9698 | Val Acc: 0.9620
Epoch 8/15 | Loss: 0.0704 | Train Acc: 0.9728 | Val Acc: 0.9620
Epoch 9/15 | Loss: 0.0671 | Train Acc: 0.9731 | Val Acc: 0.9642
Epoch 10/15 | Loss: 0.0661 | Train Acc: 0.9738 | Val Acc: 0.9688
Epoch 11/15 | Loss: 0.0575 | Train Acc: 0.9770 | Val Acc: 0.9714
Epoch 12/15 | Loss: 0.0514 | Train Acc: 0.9796 | Val Acc: 0.9512
Epoch 13/15 | Loss: 0.0505 | Train Acc: 0.9800 | Val Acc: 0.9720
Epoch 14/15 | Loss: 0.0505 | Train Acc: 0.9803 | Val Acc: 0.9698
Epoch 15/15 | Loss: 0.0454 | Train Acc: 0.9817 | Val Acc: 0.9702


In [31]:
def final_evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total


In [32]:
final_val_acc = final_evaluate(model, val_loader)
print(f"Final Validation Accuracy: {final_val_acc:.4f}")


Final Validation Accuracy: 0.9702


In [34]:
import numpy as np

def get_predictions(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    return np.array(all_preds), np.array(all_labels)



In [35]:
preds, labels = get_predictions(model, val_loader)


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [36]:
from sklearn.metrics import classification_report

print(classification_report(labels, preds, target_names=train_data.classes))


              precision    recall  f1-score   support

         cat       0.99      0.95      0.97      2500
         dog       0.96      0.99      0.97      2500

    accuracy                           0.97      5000
   macro avg       0.97      0.97      0.97      5000
weighted avg       0.97      0.97      0.97      5000



In [38]:
import os
print(os.listdir("dataset/val/cat")[:10])


['3823.jpg', '3700.jpg', '2499.jpg', '7040.jpg', '4203.jpg', '10278.jpg', '7229.jpg', '4878.jpg', '10896.jpg', '6427.jpg']


In [39]:
from PIL import Image
import torchvision.transforms as transforms

image_path = "dataset/val/cat/7229.jpg"  # use real filename

image = Image.open(image_path).convert("RGB")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

image = transform(image).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(image)
    prediction = output.argmax(1).item()

print("Predicted Class:", train_data.classes[prediction])


Predicted Class: cat
